# Sujet 2: Modélisation des interdépendances

**Quel est l’impact d’une baisse de la capacité de LNG sur le système en général et en particulier sur
le système électrique ?**

Pour aller plus loin : modéliser le stockage gaz

**Authors**: Lucas DEFRANCE, Quentin DEHAENE, Jules PAINDESSOUS--POETE

## Imports

In [1]:
# Necessary imports
using JuMP
using XLSX
using DataFrames
#use the solver you want
using HiGHS
using CSV
using Statistics

In [2]:
# Define global variables
data_file_zootopia = "data/TP_empilement.xlsx";
data_file = "data/Donnees_etude_de_cas_ETE305.xlsx";

In [27]:
#Import gaz data

V_max_gaz=XLSX.readdata(data_file, "Données gaz", "K4"); #en MwH attention
In_max_gaz=XLSX.readdata(data_file, "Données gaz", "L4"); #Mw/j
Out_max_gaz=XLSX.readdata(data_file, "Données gaz", "M4");#Mw/j
Niv_min_DtD=XLSX.readdata(data_file, "Données gaz", "P5:P368"); #A multiplier par Vmax
Niv_max_DtD=XLSX.readdata(data_file, "Données gaz", "R5:R368");#A multiplier par Vmax
Load_gaz_DtD=XLSX.readdata(data_file, "Données gaz", "C5:C370")#Mwh/j

Niv_min_HtH = [value*V_max_gaz for value in Niv_min_DtD for _ in 1:24];
Niv_max_HtH = [value*V_max_gaz for value in Niv_max_DtD for _ in 1:24];
Coeff_jour=[0.028,0.029,0.031,0.034,0.038,0.048,0.069,0.067,0.066,0.06,0.053,0.047,0.04,0.037,0.033,0.032,0.036,0.039,0.041,0.04,0.038,0.037,0.031,0.027]
println(sum(Coeff_jour))
Load_gaz_HtH=[value*Coeff_jour[j] for value in Load_gaz_DtD for j in 1:24];
println(Load_gaz_HtH[169])

coeff_import = 0.5; #coefficient d'importation à modifier si on stop la Russie
#25% d'imports russes

1.0010000000000001
3985.551910413024


In [4]:
# Extract data for power mix
data_ranges = ["V15:V17", "W15:W17", "X15:X17", "Y15:Y17", "Z15:Z17"]
column_names = [:prod_name, :pmax, :pmin, :dmin, :costs]
data_types = [String, Float64, Float64, Int, Float64]

# Read data from file
df = DataFrame()
for (col_name, range, data_type) in zip(column_names, data_ranges, data_types)
    df[!, col_name] = data_type.(vec(XLSX.readdata(data_file, "Données gaz", range)))
end


println(df)

3×5 DataFrame
 Row │ prod_name          pmax      pmin     dmin   costs   
     │ String             Float64   Float64  Int64  Float64 
─────┼──────────────────────────────────────────────────────
   1 │ Methanisation       6250.0   6000.0     168     24.0
   2 │ Imports GNL        12500.0      0.0       1    150.0
   3 │ Pyrogazeification   2083.33  1708.33    168     25.0


In [5]:
# Initialize arrays to store values for non-fatal thermal clusters
names_gaz, costs_gaz, Pmin_gaz, Pmax_gaz, dmin_gaz = [], [], [], [], []

# Loop through each row of the DataFrame
for row in eachrow(df)
 # Iterate over each plant name and unit number in the row

    # Construct the complete plant name with unit number and append it to names_th
    push!(names_gaz, row.prod_name)
    
    # Append other values to respective arrays
    push!(costs_gaz, row.costs)
    push!(Pmin_gaz, row.pmin)
    push!(Pmax_gaz, row.pmax)
    push!(dmin_gaz, row.dmin)

end

# Convert the vectors to 2D arrays
costs_gaz, Pmin_gaz, Pmax_gaz, dmin_gaz = hcat(costs_gaz), hcat(Pmin_gaz), hcat(Pmax_gaz), hcat(dmin_gaz)
println(Pmax_gaz)

# Create a dictionary to map indices to plant names
Ngaz = length(costs_gaz)
dict_gaz= Dict(i => names_gaz[i] for i in 1:length(names_gaz))

# Check the value associated with index 1 in dict_th
dict_gaz[1]

[6250.0; 12500.0; 2083.3333333333335;;]


"Methanisation"

In [6]:
# nb_week_year = 52
# UC_continu_gaz, stock_gaz_continu, UP_continu_gaz, DO_continu_gaz = [],0, [], []
# for w in 1:nb_week_year
#     # We start the simulation at Tmin and end it at Tmax
#     Tmin = (w-1)*24*7
#     Tmax = Tmin + 24*7-1; #optimization by one week intervals
#     Tp = 24*9;

      
    
#     #data for load and fatal generation
#     jour=XLSX.readdata(data_file_zootopia, "TP", "A$(10+Tmin):B$(Tp+Tmin + 10)");
#     ToD=XLSX.readdata(data_file_zootopia, "TP", "B$(10+Tmin):B$(Tp+Tmin + 10)");
#     load = XLSX.readdata(data_file_zootopia, "TP", "C$(10+Tmin):C$(Tp+Tmin + 10)");
#     wind = XLSX.readdata(data_file_zootopia, "TP", "D$(10+Tmin):D$(Tp+Tmin + 10)");
#     solar = XLSX.readdata(data_file_zootopia, "TP", "E$(10+Tmin):E$(Tp+Tmin + 10)");
#     hydro_fatal = XLSX.readdata(data_file_zootopia, "TP", "F$(10+Tmin):F$(Tp+Tmin + 10)");
#     thermal_fatal = XLSX.readdata(data_file_zootopia, "TP", "G$(10+Tmin):G$(Tp+Tmin + 10)");
#     #total of RES
#     Pres = wind + solar + hydro_fatal + thermal_fatal;
#     # println(jour)
    


#     #############################
#     #create the optimization model
#     #############################
#     model = Model(HiGHS.Optimizer)

#     #############################
#     #define the variables
#     #############################


#     #Weekly Gaz variables

#     coeff_import = 1 #coefficient d'importation à modifier si on stop la Russie
#     cgaz=repeat(costs_gaz',Tp)

#     @variable(model, P_injected_gaz[1:Tp] >= 0)
#     @variable(model, P_discharged_gaz[1:Tp] >= 0)
#     @variable(model, stock_gaz[1:Tp] >= 0)
    
#     @variable(model, 0<=P_gaz[1:Tp,1:Ngaz])
#     @variable(model, UC_gaz[1:Tp,1:Ngaz], Bin)
#     @variable(model, UP_gaz[1:Tp,1:Ngaz], Bin)
#     @variable(model, DO_gaz[1:Tp,1:Ngaz], Bin)

#     #unsupplied gaz variables
#     @variable(model, Puns_gaz[1:Tp] >= 0)
#     #in excess gaz variables
#     @variable(model, Pexc_gaz[1:Tp] >= 0)

#     cuns_gaz = 5000*ones(Tp) #cost of unsupplied gaz €/MWh
#     cexc_gaz = 0*ones(Tp); #cost of in excess gaz €/MWh


#     #############################
#     #define the objective function
#     #############################
#     @objective(model, Min, sum(P_gaz.*cgaz)+Puns_gaz'cuns_gaz+Pexc_gaz'cexc_gaz)
    


#     #############################
#     #define the constraints
#     #############################



#     #gaz stock constraint 


#     #thermal unit Pmax constraints
#     @constraint(model, max_gaz[t in 1:Tp, g in 1:Ngaz], P_gaz[t,g] <= Pmax_gaz[g]*UC_gaz[t,g])
#     #thermal unit Pmin constraints
#     @constraint(model, min_gaz[t in 1:Tp, g in 1:Ngaz], Pmin_gaz[g]*UC_gaz[t,g] <= P_gaz[t,g])

#     @constraint(model, stck_gaz_encad[t in 1:Tp], Niv_min_HtH[t]<=stock_gaz[t]<= Niv_max_HtH[t])
#     @constraint(model, stck_gaz[t in 2:Tp], stock_gaz[t] == stock_gaz[t-1]+ P_injected_gaz[t] - P_discharged_gaz[t])
#     if (w==1)
#         @constraint(model, stock_gaz[1]==2*Niv_min_HtH[1])#arbitrary choice
       
#     else
#         @constraint(model, stock_gaz[1]==stock_gaz_continu)
#     end   


#     #gaz in and out constraints
#     @constraint(model,inj_gaz[t in 1:Tp],P_injected_gaz[t]<=200000/24)#limited by pipe flow
#     @constraint(model,out_gaz_max[t in 1:Tp],P_discharged_gaz[t]<=300000/24)

#     @constraint(model, balance_gaz[t in 1:Tp], sum(P_gaz[t,g] for g in 1:Ngaz) + Puns_gaz[t] - load[t] - Pexc_gaz[t] == 0)

#     @constraint(model,out_gaz[t in 1:Tp], P_discharged_gaz[t]+ sum(P_gaz[t,g] for g in 1:Ngaz) == Load_gaz_HtH[t])
#         #Pth representent les puissances des centrqles gaz qu'il faut donc approvisonner en gaz avec un rendement



#         for g in 1:Ngaz
#             if (dmin_gaz[g] > 1)
#                 @constraint(model, [t in 2:Tp], UC_gaz[t,g]-UC_gaz[t-1,g]==UP_gaz[t,g]-DO_gaz[t,g])
#                 @constraint(model, [t in 1:Tp], UP_gaz[t]+DO_gaz[t]<=1)
#                 if (w==1)
#                     @constraint(model, UP_gaz[1,g]==0)
#                     @constraint(model, DO_gaz[1,g]==0)
#                 else
#                     @constraint(model, UC_gaz[1,g]-UC_continu_gaz[g]==UP_gaz[1,g]-DO_gaz[1,g])
#                 end   
#                 @constraint(model, [t in dmin_gaz[g]:Tp], UC_gaz[t,g] >= sum(UP_gaz[i,g] for i in (t-dmin_gaz[g]+1):t))
#                 @constraint(model, [t in dmin_gaz[g]:Tp], UC_gaz[t,g] <= 1 - sum(DO_gaz[i,g] for i in (t-dmin_gaz[g]+1):t))
#                 if (w==1)
#                     @constraint(model, [t in 1:dmin_gaz[g]-1], UC_gaz[t,g] >= sum(UP_gaz[i,g] for i in 1:t))
#                     @constraint(model, [t in 1:dmin_gaz[g]-1], UC_gaz[t,g] <= 1-sum(DO_gaz[i,g] for i in 1:t))
    
#                 else
#                     @constraint(model, [t in 1:dmin_gaz[g]-1], UC_gaz[t,g] >= sum(UP_gaz[i,g] for i in 1:t)+sum(UP_continu_gaz[j,g] for j in 168 + t - dmin_gaz[g]:168))
#                     @constraint(model, [t in 1:dmin_gaz[g]-1], UC_gaz[t,g] <= 1-sum(DO_gaz[i,g] for i in 1:t)-sum(DO_continu_gaz[j,g] for j in 168 + t - dmin_gaz[g]:168))
#                 end
#             end
#         end 
    

    
#     #no need to print the model when it is too big
#     #solve the model
#     optimize!(model)
    
#     # Collecting values for each iteration

#     UC_continu_gaz= value.(UC_gaz[Tp-48, 1:Ngaz])
#     UP_continu_gaz= value.(UP_gaz[Tp-215:Tp-48, 1:Ngaz])
#     DO_continu_gaz= value.(DO_gaz[Tp-215:Tp-48, 1:Ngaz])
#     stock_gaz_continu= value(stock_gaz[Tp-48])

    
#     #------------------------------
#     #Results
#     # println("semaine :",w)
#     @show termination_status(model)
#     # @show objective_value(model)

#     #exports results as csv file
#     # th_gen = value.(Pth)
#     # hy_gen = value.(Phy)
#     # STEP_charge = value.(Pcharge_STEP)
#     # STEP_decharge = value.(Pdecharge_STEP)
#     gaz_gen=value.(P_gaz)

#     # new file created
#     file_name = string("results_avec_gaz/justgazresultsW",w,".csv")
#     touch(file_name)

#     # file handling in write mode
#     f = open(file_name, "w")

#     write(f, "Jour, Heure, Demande_gaz,")
#     for i in 1:length(dict_gaz)
#         plant_name = dict_gaz[i]
#         write(f, string(plant_name, ","))
#     end
#     write(f,'\n')
    

    
#     for t in 1:Tp-48
#         write(f, "$(jour[t]),  $(ToD[t]), $(Load_gaz_HtH[t]),")
#         for g in 1:Ngaz
#             write(f, "$(gaz_gen[t,g]), ")
#         end
#         write(f,'\n')
#     end

    

#     close(f)
# end

In [28]:
nb_week_year = 52
UC_continu, stock_STEP_continu, UP_continu, DO_continu = [], [], [], []
UC_continu_gaz, stock_gaz_continu, UP_continu_gaz, DO_continu_gaz = [],0, [], []
sizeUP_gaz=0
Load_gaz_sem=[]
for w in 1:nb_week_year
    # We start the simulation at Tmin and end it at Tmax
    Tmin = (w-1)*24*7
    Tmax = Tmin + 24*7-1; #optimization by one week intervals
    Tp = 24*9;

      
    
    #data for load and fatal generation
    jour=XLSX.readdata(data_file_zootopia, "TP", "A$(10+Tmin):B$(Tp+Tmin + 10)");
    ToD=XLSX.readdata(data_file_zootopia, "TP", "B$(10+Tmin):B$(Tp+Tmin + 10)");
    load = XLSX.readdata(data_file_zootopia, "TP", "C$(10+Tmin):C$(Tp+Tmin + 10)");
    wind = XLSX.readdata(data_file_zootopia, "TP", "D$(10+Tmin):D$(Tp+Tmin + 10)");
    solar = XLSX.readdata(data_file_zootopia, "TP", "E$(10+Tmin):E$(Tp+Tmin + 10)");
    hydro_fatal = XLSX.readdata(data_file_zootopia, "TP", "F$(10+Tmin):F$(Tp+Tmin + 10)");
    thermal_fatal = XLSX.readdata(data_file_zootopia, "TP", "G$(10+Tmin):G$(Tp+Tmin + 10)");
    
    Load_gaz=Load_gaz_HtH[Tmin+1:Tmin+Tp]

    #GAZ VARIABLES
    model = Model(HiGHS.Optimizer)
    
    cgaz=repeat(costs_gaz',Tp)

    @variable(model, P_injected_gaz[1:Tp] >= 0)
    @variable(model, P_discharged_gaz[1:Tp] >= 0)
    @variable(model, stock_gaz[1:Tp] >= 0)
    
    @variable(model, 0<=P_gaz[1:Tp,1:Ngaz])
    @variable(model, UC_gaz[1:Tp,1:Ngaz], Bin)
    @variable(model, UP_gaz[1:Tp,1:Ngaz], Bin)
    @variable(model, DO_gaz[1:Tp,1:Ngaz], Bin)

    #unsupplied gaz variables
    @variable(model, Puns_gaz[1:Tp] >= 0)
    #in excess gaz variables
    @variable(model, Pexc_gaz[1:Tp] >= 0)

    cuns_gaz = 5000*ones(Tp) #cost of unsupplied gaz €/MWh
    cexc_gaz = 1*ones(Tp); #cost of in excess gaz €/MWh
    

    #############################
    #define the objective function
    #############################

    @objective(model, Min, sum(P_gaz.*cgaz)+Puns_gaz'cuns_gaz+Pexc_gaz'cexc_gaz)
    


    #GAZ CONSTRAINTS
    @constraint(model, stck_gaz_encad[t in 1:Tp], Niv_min_HtH[t]<=stock_gaz[t]<= Niv_max_HtH[t])
    @constraint(model, stck_gaz[t in 2:Tp], stock_gaz[t] == stock_gaz[t-1]+ P_injected_gaz[t] - P_discharged_gaz[t])
    if (w==1)
        @constraint(model, stock_gaz[1]==0.599*V_max_gaz)#arbitrary choice to start at mean level
       
    else
        @constraint(model, stock_gaz[1]==stock_gaz_continu)
    end   

    #thermal unit Pmax constraints
    @constraint(model, max_gaz_meth[t in 1:Tp], P_gaz[t,1] <= Pmax_gaz[1]*UC_gaz[t,1])
    @constraint(model, max_gaz_gnl[t in 1:Tp], P_gaz[t,2] <= Pmax_gaz[2]*UC_gaz[t,2]*coeff_import)#controlling how much we can import 
    @constraint(model, max_gaz_pyro[t in 1:Tp], P_gaz[t,3] <= Pmax_gaz[3]*UC_gaz[t,3])

    #thermal unit Pmin constraints
    @constraint(model, min_gaz[t in 1:Tp, g in 1:Ngaz], Pmin_gaz[g]*UC_gaz[t,g] <= P_gaz[t,g])

    #gaz in and out constraints
    @constraint(model,inj_gaz[t in 1:Tp],P_injected_gaz[t]<=In_max_gaz/24)#limited by pipe flow
    @constraint(model,out_gaz_max[t in 1:Tp],P_discharged_gaz[t]<=Out_max_gaz/24)

    # Load_elec_gaz=[0 for i in 1:Tp]
    # (Load_elec_gaz[t]=(Pth[t,7]+Pth[t,8]+Pth[t,9]+Pth[t,10]+Pth[t,11]+Pth[t,12]+Pth[t,13])/0.6 + (Pth[t,14]+Pth[t,15])/0.4 + (Pth[t,16]+Pth[t,17]+Pth[t,18])/0.5 for t in 1:Tp)
    #Pth representent les puissances des centrales gaz qu'il faut donc approvisonner en gaz avec un rendement

    @constraint(model,out_gaz[t in 1:Tp], P_discharged_gaz[t]+ sum(P_gaz[t,g] for g in 1:Ngaz) + Puns_gaz[t]- Pexc_gaz[t]
    -(P_injected_gaz[t]+Load_gaz[t])==0)
    



    for g in 1:Ngaz
        if (dmin_gaz[g] > 1)
            @constraint(model, [t in 2:Tp], UC_gaz[t,g]-UC_gaz[t-1,g]==UP_gaz[t,g]-DO_gaz[t,g])
            @constraint(model, [t in 1:Tp], UP_gaz[t,g]+DO_gaz[t,g]<=1)
            if (w==1)
                @constraint(model, UP_gaz[1,g]==0)
                @constraint(model, DO_gaz[1,g]==0)
            else
                @constraint(model, UC_gaz[1,g]-UC_continu_gaz[g]==UP_gaz[1,g]-DO_gaz[1,g])
            end   
            @constraint(model, [t in dmin_gaz[g]:Tp], UC_gaz[t,g] >= sum(UP_gaz[i,g] for i in (t-dmin_gaz[g]+1):t))
            @constraint(model, [t in dmin_gaz[g]:Tp], UC_gaz[t,g] <= 1 - sum(DO_gaz[i,g] for i in (t-dmin_gaz[g]+1):t))
            if (w==1)
                @constraint(model, [t in 1:dmin_gaz[g]-1], UC_gaz[t,g] >= sum(UP_gaz[i,g] for i in 1:t))
                @constraint(model, [t in 1:dmin_gaz[g]-1], UC_gaz[t,g] <= 1-sum(DO_gaz[i,g] for i in 1:t))
    
            else
                @constraint(model, [t in 1:dmin_gaz[g]-1], UC_gaz[t,g] >= sum(UP_gaz[i,g] for i in 1:t)+sum(UP_continu_gaz[j,g] for j in sizeUP_gaz + t - dmin_gaz[g]:sizeUP_gaz))
                @constraint(model, [t in 1:dmin_gaz[g]-1], UC_gaz[t,g] <= 1-sum(DO_gaz[i,g] for i in 1:t)-sum(DO_continu_gaz[j,g] for j in sizeUP_gaz + t - dmin_gaz[g]:sizeUP_gaz))
            end
        
        end
    end 
    

    

    #no need to print the model when it is too big
    #solve the model
    optimize!(model)
    
    # Collecting values for each iteration
    
    UC_continu_gaz= value.(UC_gaz[Tp-48, 1:Ngaz])
    UP_continu_gaz= value.(UP_gaz[Tp-215:Tp-48, 1:Ngaz])
    DO_continu_gaz= value.(DO_gaz[Tp-215:Tp-48, 1:Ngaz])
    stock_gaz_continu= value(stock_gaz[Tp-48])
    sizeUP_gaz=size(UP_continu_gaz,1)
    println(sizeUP_gaz)
    
    #------------------------------
    #Results
    # println("semaine :",w)
    @show termination_status(model)
    # @show objective_value(model)

    #exports results as csv file
    gaz_gen=value.(P_gaz)
    gaz_stock_in=value.(P_injected_gaz)
    gaz_stock_discharged=value.(P_discharged_gaz)
    gaz_uns=value.(Puns_gaz)
    gaz_exec=value.(Pexc_gaz)

    # new file created
    file_name = string("results_avec_gaz/only_gaz_resultsW",w,".csv")
    touch(file_name)

    # file handling in write mode
    f = open(file_name, "w")

    write(f, "Jour, Heure,")
    # for i in 1:length(dict_th)
    #     plant_name = dict_th[i]
    #     write(f, string(plant_name, ","))
    # end
    # write(f,"Hydro, STEP pompage, STEP turbinage,")

    write(f, "Demande en CH4,")
    for i in 1:length(dict_gaz)
        plant_name = dict_gaz[i]
        write(f, string(plant_name, ","))
    end
    write(f, "Stock Gaz in, Stock Gaz out, P Uns Gaz, P Exc Gaz \n")
   
    

    
    for t in 1:Tp-48
        write(f,'\n')
        write(f, "$(jour[t]), $(ToD[t]),")
        # for g in 1:Nth
        #     write(f, "$(th_gen[t,g]), ")
        # end
        # for h in 1:Nhy
        #     write(f, "$(hy_gen[t,h]),")
        # end
        # write(f, "$(STEP_charge[t]), $(STEP_decharge[t]),")
        write(f,"$(Load_gaz[t]), ")
        for g in 1:Ngaz
            write(f, "$(gaz_gen[t,g]), ")
        
        end
        
        write(f,"$(gaz_stock_in[t]),$(gaz_stock_discharged[t]),$(gaz_uns[t]),$(gaz_exec[t]) ")
        
    
        
    end
    

    close(f)
end

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
3231 rows, 3019 cols, 96123 nonzeros
2606 rows, 2610 cols, 97689 nonzeros
2281 rows, 2151 cols, 102038 nonzeros
2279 rows, 2150 cols, 102034 nonzeros

Solving MIP model with:
   2279 rows
   2150 cols (1051 binary, 0 integer, 0 implied int., 1099 continuous)
   102034 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   -34055077.7066  inf                  inf        0      0      0         0     1.2s
 T       0       0         0   0.00%   -34055077.7066  0                  Large        0      0      0       284     1.2s

Solving report
  Status            Optimal
  Primal bound      0
  Dual bound        0
  Gap               0% (tolerance: 0.01%)
  Solution stat